In [1]:
#Load the libraries
import os 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set the working directory
path = "C:/Users/keyur/Downloads"
os.chdir(path)

In [3]:
# Load data into a Pandas DataFrame
well_data = pd.read_csv('well_data.csv')
well_data.head()

,StateWellNumber,ParameterDescription_x,TOTAL DISSOLVED SOLIDS,wkt_geom,County,AquiferCode,WellDepth,Date,Time,SampleNumber,EntityId,LabId,ReliabilityId,CollectionRemarks,LatitudeDD,LongitudeDD,ParameterCode,ParameterDescription_y,ARSENIC,ParameterUnit
0,1204617,"TOTAL DISSOLVED SOLIDS , SUM OF CONSTITUENTS (...",1.296,Point (-100.51444499999999493 34.9202780000000...,Collingsworth,110ALVM,228,5/7/2010,815,True,1,23,7,Analysis Balanced. Lab Calculated Anion/Cation...,34.920278,-100.514445,1000,"ARSENIC, DISSOLVED (UG/L AS AS)",4.40,ug/L
1,1205911,"TOTAL DISSOLVED SOLIDS , SUM OF CONSTITUENTS (...",0.399,Point (-100.39722220000000164 34.9033332999999...,Collingsworth,110ALVM,90,7/29/2015,1615,True,1,24,7,Analysis Unbalanced.,34.903333,-100.397222,1000,"ARSENIC, DISSOLVED (UG/L AS AS)",2.00,ug/L
2,1206707,"TOTAL DISSOLVED SOLIDS , SUM OF CONSTITUENTS (...",0.787,Point (-100.33416699999999366 34.8933340000000...,Collingsworth,110ALVP,110,7/25/2006,1712,True,1,24,7,Analysis Balanced. Lab Calculated Anion/Cation...,34.893334,-100.334167,1000,"ARSENIC, DISSOLVED (UG/L AS AS)",4.10,ug/L
3,1206918,"TOTAL DISSOLVED SOLIDS , SUM OF CONSTITUENTS (...",0.498,Point (-100.28694439999999588 34.8900000000000...,Collingsworth,110ATSB,0,7/29/2015,1720,True,1,23,7,Analysis Balanced. Lab Calculated Anion/Cation...,34.890000,-100.286944,1000,"ARSENIC, DISSOLVED (UG/L AS AS)",2.31,ug/L
4,1215111,"TOTAL DISSOLVED SOLIDS , SUM OF CONSTITUENTS (...",0.743,Point (-100.23416699999999935 34.8513889999999...,Collingsworth,110ALVM,120,5/5/2010,1245,True,1,23,7,Analysis Balanced. Lab Calculated Anion/Cation...,34.851389,-100.234167,1000,"ARSENIC, DISSOLVED (UG/L AS AS)",5.00,ug/L


In [4]:
# create new column Exceedance and set values based on condition
well_data['Exceedance'] = well_data['ARSENIC'].apply(lambda x: 1 if x >= 3 else 0) #Use thresold of 3 

In [5]:
# Set the dependent variable (Arsenic--exceeding the limit or not)
y = well_data['Exceedance']

In [7]:
# Select the independent variables
X = well_data[['TOTAL DISSOLVED SOLIDS','WellDepth']]

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
# Fit the Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)

In [10]:
# Test the model on the test set & check for accuracy and confusion matrix
lr_accuracy = lr_model.score(X_test, y_test) 
lr_cmatrix = confusion_matrix(y_test, lr_preds) 
print('LR Model Accuracy:', lr_accuracy)
print("Logistic Regression Model Confusion Matrix:\n", lr_cmatrix)

LR Model Accuracy: 0.6190476190476191
Logistic Regression Model Confusion Matrix:
 [[10  2]
 [ 6  3]]


In [11]:
# Fit the NBC model
nbc_model = GaussianNB()
nbc_model.fit(X_train, y_train)
nbc_preds = nbc_model.predict(X_test)

In [12]:
# Test the model on the test set & check for accuracy and confusion matrix
nbc_accuracy = nbc_model.score(X_test, y_test)
nb_cmatrix = confusion_matrix(y_test, nbc_preds)
print('NBC Model Accuracy:', nbc_accuracy)
print("Naive Bayes Model Confusion Matrix:\n", nb_cmatrix)

NBC Model Accuracy: 0.7619047619047619
Naive Bayes Model Confusion Matrix:
 [[12  0]
 [ 5  4]]


In [13]:
# Create a grid of uniformly spaced points over the domain of the aquifer
x = np.linspace(min(X['TOTAL DISSOLVED SOLIDS']), max(X['TOTAL DISSOLVED SOLIDS']), 50)
y = np.linspace(min(X['WellDepth']), max(X['WellDepth']), 50)
xx, yy = np.meshgrid(x, y)
xy = np.vstack([xx.ravel(), yy.ravel()]).T

In [14]:
# Use the LR and NBC models to predict vulnerability at each point in the grid
lr_pred = lr_model.predict(xy)
nbc_pred = nbc_model.predict(xy)

To be continued...